In [1]:
import os 
import json
import glob
import pandas as pd
outdir = 'results/main/hubs/loop_hub/'
link_path = 'https://informaticsdata.liai.org/BioAdHoc/Groups/vd-ay/jreyna/projects/dchallenge/results/main/hubs/loop_hub/'
os.makedirs(outdir, exist_ok=True)

In [2]:
os.chdir('/mnt/BioHome/jreyna/jreyna/projects/dchallenge/')

In [3]:
# setting template names
loop_tpl = 'results/main/h3k27ac_hichip/*/FitHiChIP_S/FitHiChIP.interactions_FitHiC.track.longrange.bed.gz'

## Getting all WashU Files

#### Loop Files

In [4]:
loop_data = []
for loop in glob.glob(loop_tpl):
    cline = loop.split('/')[3]
    loop_data.append([cline, loop])
loop_df = pd.DataFrame(loop_data)
loop_df.columns = ['cline', 'file']

In [5]:
# # loop_short_list = ['CD4N',
# #                  'CD8N',
# #                  'NB',
# #                  'CM',
# #                  'NK']
# loop_df = loop_df.loc[loop_df.cline.isin(loop_short_list)]

In [6]:
loop_df.shape

(5, 2)

In [7]:
loop_df

,cline,file
0,NK-cell_naive,results/main/h3k27ac_hichip/NK-cell_naive/FitH...
1,CD8_T-cell_naive,results/main/h3k27ac_hichip/CD8_T-cell_naive/F...
2,CD4_T-cell_naive,results/main/h3k27ac_hichip/CD4_T-cell_naive/F...
3,monocyte_naive,results/main/h3k27ac_hichip/monocyte_naive/Fit...
4,B-cell_naive,results/main/h3k27ac_hichip/B-cell_naive/FitHi...


## Making the Hub

In [8]:
loop_cline_abbrevs = [['CD4N', 'CD4T-cell'],
                         ['NK', 'NK-cell'],
                         ['CD8N', 'CD8T-cell'],
                         ['CM', 'Monocyte'],
                         ['NB', 'B-cell']]
loop_cline_abbrevs = pd.DataFrame(loop_cline_abbrevs)
loop_cline_abbrevs.columns = ['loop_cline', 'abbrev_cline']
#loop_cline_abbrevs.set_index('loop_cline', inplace=True)
loop_cline_abbrevs

,loop_cline,abbrev_cline
0,CD4N,CD4T-cell
1,NK,NK-cell
2,CD8N,CD8T-cell
3,CM,Monocyte
4,NB,B-cell


In [9]:
def create_washu_longrange_dict(name, file):
    d = {"type": "longrange",
         "showOnHubLoad": True,
         "name": name,
         "filename": file,
         "options": {"displayMode": "arc"}}
    return(d)

def create_washu_bed_dict(name, file):
    d = {"type": "bed",
         "showOnHubLoad": True,
         "name": name,
         "filename": file}
    return(d)

def create_washu_vcf_dict(name, file):
    d = {"type": "vcf",
         "showOnHubLoad": True,
         "name": name,
         "filename": file}
    return(d)

In [13]:
def create_remote_washu_longrange_dict(name, url):
    d = {'type': 'longrange',
         'showOnHubLoad': True,
         'name': name,
         'url': url,
         'options': {'ensemblStyle': True, 'height': 100, 'displayMode': 'arc'}}
    return(d)

In [11]:
json_files = []

for i, sr in loop_df.iterrows():
    
    print(sr.cline)

    new_basename = 'loops.{}.bed.gz'.format(sr.cline)

    # hard link over the file
    old_path = sr.file
    old_index_path = sr.file + '.tbi'
    new_path = os.path.join(outdir, new_basename)
    new_index_path = new_path + '.tbi'

    if os.path.exists(old_path) and os.path.exists(old_index_path):
        if not os.path.exists(new_path) and not os.path.exists(new_index_path):
            os.link(old_path, new_path)
            os.link(old_index_path, new_index_path)

        url = os.path.join(link_path, new_basename)

        # add the file to the json hub config
        d = create_remote_washu_longrange_dict(new_basename, url)
        json_files.append(d)

                
hub_path = os.path.join(outdir, 'hub.config.json')
with open(hub_path, 'w') as fw:
    print(json.dumps(json_files, indent='\t'), file=fw)

NK-cell_naive
CD8_T-cell_naive
CD4_T-cell_naive
monocyte_naive
B-cell_naive
